In [8]:
import utils
import os
import openai
openai.api_key = utils.get_openai_api_key()

AttributeError: module 'utils' has no attribute 'get_openai_api_key'

In [10]:
import os
from dotenv import load_dotenv, find_dotenv

import numpy as np
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)

AttributeError: module 'utils' has no attribute 'get_openai_api_key'

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [4]:
from llama_index.core.llms import LLM
from llama_index.llms.openai import OpenAI

In [19]:
import trulens_eval.feedback as fb

In [20]:
print(dir(fb))

['AzureOpenAI', 'Bedrock', 'Embeddings', 'Feedback', 'GroundTruthAgreement', 'Huggingface', 'Langchain', 'LiteLLM', 'OpenAI', 'OptionalImports', 'REQUIREMENT_BEDROCK', 'REQUIREMENT_LITELLM', 'REQUIREMENT_OPENAI', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'embeddings', 'feedback', 'groundtruth', 'mod_feedback', 'prompts', 'provider', 'v2']


In [22]:
from trulens_eval.feedback import Embeddings

In [13]:
import trulens_eval

In [21]:
import nest_asyncio